# Material Models


## Shells


In [56]:
from sigmaepsilon.solid.material import MindlinShellSection as Section

section = Section(
    layers=[Section.Layer(material=Section.Material(E=2100000, nu=0.3), thickness=0.1)]
)
section.elastic_stiffness_matrix()

array([[2.30769231e+05, 6.92307692e+04, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.92307692e+04, 2.30769231e+05, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 8.07692308e+04, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.92307692e+02,
        5.76923077e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.76923077e+01,
        1.92307692e+02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 6.73076923e+01, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 6.73076923e+04, 0.

## Membranes


A membrane specimen can be constructed as a list of layers, each defined with its own elasticity tensor.


In [57]:
from sigmaepsilon.solid.material import MembraneSection as Section
from sigmaepsilon.math.linalg import ReferenceFrame
from sigmaepsilon.solid.material import ElasticityTensor
from sigmaepsilon.solid.material.utils import elastic_stiffness_matrix
import numpy as np

yield_strength = 355.0
hooke = elastic_stiffness_matrix(E=210000, NU=0.3)
frame = ReferenceFrame(dim=3)
tensor = ElasticityTensor(
    hooke, frame=frame, tensorial=False, yield_strength=yield_strength
)

section = Section(layers=[Section.Layer(material=tensor, thickness=0.1)])
section.elastic_stiffness_matrix()

array([[28269.23076923, 12115.38461538,     0.        ],
       [12115.38461538, 28269.23076923,     0.        ],
       [    0.        ,     0.        ,  8076.92307692]])

In [58]:
section.utilization(strains=2 * np.random.rand(3) / 1000) * 100

83.46916296091503

In [59]:
section.calculate_equivalent_stress(strains=np.array([1.0, 0.0, 0.0]))

array(245650.02526204)

In [60]:
section.calculate_equivalent_stress(strains=np.array([1.0, 0.0, 0.0]), z=[-1, 0, 1])

array([245650.02526204, 245650.02526204, 245650.02526204])

In [61]:
section.calculate_equivalent_stress(strains=np.array([1.0, 0.0, 0.0]), z=[0])

array(245650.02526204)

In [62]:
section.calculate_equivalent_stress(
    strains=np.array([1.0, 0.0, 0.0]), z=[0], squeeze=False
)

array([[245650.02526204]])

In [63]:
section.calculate_equivalent_stress(
    strains=np.array([1.0, 0.0, 0.0]), z=[-1, 0, 1]
).shape

(3,)

In [64]:
section.utilization(strains=np.array([1.0, 0.0, 0.0]), z=[0], squeeze=False)

array([[691.97190215]])

In [65]:
section.utilization(strains=np.array([1.0, 0.0, 0.0]), z=[0], squeeze=True)

array(691.97190215)

In [66]:
section.utilization(strains=np.array([1.0, 0.0, 0.0]), z=[-1, 0, 1], squeeze=False)

array([[691.97190215, 691.97190215, 691.97190215]])

In [67]:
section.utilization(strains=np.array([1.0, 0.0, 0.0]), z=[-1, 0, 1], squeeze=True)

array([691.97190215, 691.97190215, 691.97190215])

In [68]:
section.utilization(strains=2 * np.random.rand(5, 3) / 1000) * 100

array([219.0847106 , 138.63217248, 210.65448794,  46.99779752,
       198.70707613])

In [69]:
section.calculate_equivalent_stress(strains=2 * np.random.rand(5, 3) / 1000)

array([420.45281744, 539.21565063, 431.70665545, 800.7643579 ,
       447.35298778])

### Creating an invalid membrane


The `MembraneSection` class allows for definitions that has other than membrane stiffness, but a warning is thrown at creating an instance if it happens to be so. For example, if we create a membrane out of 3 layers of equal thicknessm but one having significantly smaller elastic modulus couples strenthing and bending response of a specimen.


In [70]:
from sigmaepsilon.solid.material import MembraneSection as Section

section = Section(
    layers=[
        Section.Layer(material=Section.Material(E=2100000, nu=0.3), thickness=0.1),
        Section.Layer(material=Section.Material(E=2100000, nu=0.3), thickness=0.1),
        Section.Layer(material=Section.Material(E=210000, nu=0.3), thickness=0.1),
    ]
)
section.elastic_stiffness_matrix()

F:\GitHub\SigmaEpsilon\sigmaepsilon.solid.material\src\sigmaepsilon\solid\material\surface\membrane.py:109: SigmaEpsilonMaterialWarning: [sigmaepsilon.solid.material] It seems that handling this section as a membrane only would result in missing out on the effect of bending-extension coupling. It is suggested to use a shell instead.
  warnings.warn(


array([[484615.38461538, 145384.61538462,      0.        ],
       [145384.61538462, 484615.38461538,      0.        ],
       [     0.        ,      0.        , 169615.38461538]])

## Plates


In [71]:
from sigmaepsilon.solid.material import MindlinPlateSection as Section

section = Section(
    layers=[
        Section.Layer(material=Section.Material(E=2100000, nu=0.3), thickness=0.1),
        Section.Layer(material=Section.Material(E=2100000, nu=0.3), thickness=0.1),
        Section.Layer(material=Section.Material(E=2100000, nu=0.3), thickness=0.1)
    ]
)
section.elastic_stiffness_matrix()

array([[  5192.30769231,   1557.69230769,      0.        ,
             0.        ,      0.        ],
       [  1557.69230769,   5192.30769231,      0.        ,
             0.        ,      0.        ],
       [     0.        ,      0.        ,   1817.30769231,
             0.        ,      0.        ],
       [     0.        ,      0.        ,      0.        ,
        201923.07692308,      0.        ],
       [     0.        ,      0.        ,      0.        ,
             0.        , 201923.07692308]])

In [72]:
section.layers[0]._shear_factors_x

array([0.        , 2.77777778, 4.44444444])

In [73]:
section.layers[0]._shear_factors_y

array([0.        , 2.77777778, 4.44444444])

In [74]:
section.layers[2]._loc_to_shear_factors(0.0)

(2.777777777777776, 2.7777777777777795)

In [75]:
np.stack([section.layers[0]._shear_factors_x, section.layers[0]._shear_factors_y], axis=1)

array([[0.        , 0.        ],
       [2.77777778, 2.77777778],
       [4.44444444, 4.44444444]])

In [76]:
np.stack([section.layers[1]._shear_factors_x, section.layers[1]._shear_factors_y], axis=1)

array([[4.44444444, 4.44444444],
       [5.        , 5.        ],
       [4.44444444, 4.44444444]])

In [77]:
np.stack([section.layers[2]._shear_factors_x, section.layers[2]._shear_factors_y], axis=1)

array([[4.44444444, 4.44444444],
       [2.77777778, 2.77777778],
       [0.        , 0.        ]])

In [78]:
from sigmaepsilon.solid.material import MindlinPlateSection as Section
from sigmaepsilon.math.linalg import ReferenceFrame
from sigmaepsilon.solid.material import ElasticityTensor
from sigmaepsilon.solid.material.utils import elastic_stiffness_matrix
import numpy as np

yield_strength = 355.0
hooke = elastic_stiffness_matrix(E=210000, NU=0.3)
frame = ReferenceFrame(dim=3)
tensor = ElasticityTensor(
    hooke, frame=frame, tensorial=False, yield_strength=yield_strength
)

section = Section(
    layers=[
        Section.Layer(material=tensor, thickness=0.1),
        Section.Layer(material=tensor, thickness=0.1),
    ]
)
section.elastic_stiffness_matrix().shape

(5, 5)

In [79]:
section.calculate_equivalent_stress(strains=np.array([1, 0, 0, 0, 0]), z=np.array([-0.5]))

array(12282.5012631)

In [80]:
section.calculate_equivalent_stress(strains=np.array([1, 0, 0, 0, 0]), z=np.array([-0.05, 0.05])).shape

(2,)

In [81]:
section.calculate_equivalent_stress(strains=np.array([1, 0, 0, 0, 0])).shape

(2, 3)

In [82]:
section.calculate_equivalent_stress(strains=2 * np.random.rand(10, 5) / 1000).shape

(10, 2, 3)

In [83]:
section.calculate_equivalent_stress(
    strains=2 * np.random.rand(10, 5) / 1000, z=np.array([-0.5, 0.0, 0.5])
).shape

(10, 3)

In [84]:
section.calculate_equivalent_stress(
    strains=2 * np.random.rand(10, 5) / 1000, z=np.linspace(-1, 1, 10)
).shape

(10, 10)

In [85]:
section = Section(
    layers=[
        Section.Layer(material=Section.Material(E=2100000, nu=0.3), thickness=0.1),
        Section.Layer(material=Section.Material(E=2100000, nu=0.3), thickness=0.1),
        Section.Layer(
            material=Section.Material(E=210000, nu=0.3), thickness=0.1, angle=30.0
        ),
    ]
)
section.elastic_stiffness_matrix()

F:\GitHub\SigmaEpsilon\sigmaepsilon.solid.material\src\sigmaepsilon\solid\material\surface\mindlin.py:282: SigmaEpsilonMaterialWarning: [sigmaepsilon.solid.material] It seems that handling this section as a plate only would result in missing out on the effect of bending-extension coupling. It is suggested to use a shell instead.
  warnings.warn(


array([[ 2.94230769e+03,  8.82692308e+02,  1.34506306e-14,
         0.00000000e+00,  0.00000000e+00],
       [ 8.82692308e+02,  2.94230769e+03,  1.41373753e-14,
         0.00000000e+00,  0.00000000e+00],
       [ 1.34506306e-14,  1.41373753e-14,  1.02980769e+03,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.39991420e+05, -2.92123253e-13],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -2.92123253e-13,  1.39991420e+05]])

In [86]:
section = Section(
    layers=[
        Section.Layer(material=Section.Material(E=2100000, nu=0.3), thickness=0.1),
        Section.Layer(
            material=Section.Material(
                E1=2100000,
                E2=5000,
                E3=10,
                nu12=0.3,
                G13=3000.0,
                isoplane="12",
            ),
            thickness=0.1,
            angle=30.0,
        ),
    ]
)
section.elastic_stiffness_matrix()

F:\GitHub\SigmaEpsilon\sigmaepsilon.solid.material\src\sigmaepsilon\solid\material\surface\mindlin.py:282: SigmaEpsilonMaterialWarning: [sigmaepsilon.solid.material] It seems that handling this section as a plate only would result in missing out on the effect of bending-extension coupling. It is suggested to use a shell instead.
  warnings.warn(


array([[ 1.53846147e+03,  4.61538385e+02,  8.96196946e-06,
         0.00000000e+00,  0.00000000e+00],
       [ 4.61538385e+02,  1.53846147e+03, -8.96196939e-06,
         0.00000000e+00,  0.00000000e+00],
       [ 8.96196946e-06, -8.96196939e-06,  5.38461554e+02,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         9.96299487e+02, -8.45624608e-15],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -8.45624608e-15,  1.34615385e+05]])